# Launch a Seldon Deployment
> Get an ML endpoint up and running on your cluster!

- toc: true 
- badges: true
- comments: true
- categories: [kubernetes, docker]

### Intro
In this post, we will walkthrough the first steps towards launching your own seldon deployment! 

### Launch cluster
To get started, let's get a cluster up and running. If you followed [part 1]() of this series, you can do so with kind. 
Below, I create a cluster, create a namespace called seldon-intro, then use `kubens` to make seldon-into my default namespace (so I don't have to include it in every command).

In [ ]:
!kind create cluster 
!kubectl create namespace seldon-intro
!kubens seldon-intro

#### Install seldon-core 
To install seldon-core on the cluster, use helm. To install helm itself, find directions [here](https://helm.sh/), or use `brew install helm` on mac.

Once helm is installed, use it to install seldon-core and seldon-core-operator with the following command:

In [ ]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon-intro 

To check the install is running correctly, run the following command: 

In [ ]:
!kubectl get deployments, pods

You should see a pod and deployment with `seldon-controller-manager` in the name. This pod and deployment house the seldon-core operator, which is an extensions to to the kubernetes api that allows us to deploy seldon inference graphs later on. We don't need to worry much about these details for now. 

### Build example
I'm taking this example code directly from [seldon-core irisClassifier example](https://github.com/SeldonIO/seldon-core/blob/master/examples/models/sklearn_iris/sklearn_iris.ipynb). 
This is a classic sklearn example we will be able to get up quick. 

In [2]:
!mkdir iris_classifier

mkdir: iris_classifier: File exists


In [3]:
%%writefile iris_classifier/train_iris.py
import joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import datasets


OUTPUT_FILE = "iris_classifier/IrisClassifier.sav"


def main():
    clf = LogisticRegression(solver="liblinear", multi_class="ovr")
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    print(f"Saving model in {OUTPUT_FILE}")
    joblib.dump(p, OUTPUT_FILE)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")

    main()

Writing iris_classifier/train_iris.py


In [4]:
!python iris_classifier/train_iris.py

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


In [5]:
%%writefile iris_classifier/IrisClassifier.py
import joblib

class IrisClassifier(object):

    def __init__(self):
        self.model = joblib.load('IrisClassifier.sav')

    def predict(self,X,features_names):
        return self.model.predict_proba(X)

Writing iris_classifier/IrisClassifier.py


I'm going to slightly differ from their example, and use a Dockerfile to create the docker image for this component instead of s2i. Feel free to use s2i directly from their example instead! 

In [8]:
%%writefile iris_classifier/requirements.txt
sklearn
seldon-core

Writing iris_classifier/requirements.txt


In [26]:
%%writefile iris_classifier/Dockerfile
FROM python:3.7-slim
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
EXPOSE 5000

# Define environment variable
ENV MODEL_NAME IrisClassifier 
ENV API_TYPE REST
ENV SERVICE_TYPE MODEL 
ENV PERSISTENCE 0

# seldon-core-microservice is a command line tool installed with the seldon-core python libray. You can use this locally as well!
CMD exec seldon-core-microservice $MODEL_NAME $API_TYPE --service-type $SERVICE_TYPE --persistence $PERSISTENCE

Overwriting iris_classifier/Dockerfile


To test this example, let's build and run the docker image! 

In [27]:
!docker build iris_classifier/ -t localhost:5000/iris_ex:latest

Sending build context to Docker daemon  10.24kB
Step 1/10 : FROM python:3.7-slim
 ---> b386e7420fc3
Step 2/10 : COPY . /app
 ---> 76c42605d763
Step 3/10 : WORKDIR /app
 ---> Running in 85636b5172e2
Removing intermediate container 85636b5172e2
 ---> f4bf613a6752
Step 4/10 : RUN pip install -r requirements.txt
 ---> Running in a899f52db252
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=d57a3d7fc8fdc05876bf438f26f8cff4722b0a05138d2bed022fee0a4178e0f3
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-py3-none-any.whl size=9070 sha256=a9eea2227c6a62575da5bb4ceff30ac5e17aeb30ecd101c439ecf4159c8d5e8d
  Stored in directory: /root/.cache/pip/wheels/42/22/cd/ccb93fa68f4a01fb6c10082f97bcb2af9eb8e43565ce38a292
  Created wheel for jaeger-client: filename=jaeger_client-4.3.0-py3-none-any.whl size=64291 sha256=dc0fbf6075563689924d174

In [28]:
# !docker push localhost:5000/iris_ex:latest

In [29]:
!docker run --name "iris_predictor1" -d --rm -p 5001:5000 localhost:5000/iris_ex:latest

5eec4505001959617aa301ca764808ba6921abffb450cfcdbed0139e3d74b705


You could also remove the -d argument from the above command and run this command in a separate window to see the log output while sending requests to the endpoint. Test the endpoint with the curl below! 

In [32]:
!curl  -s http://localhost:5001/predict -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'
        

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364059,0.04505474761562512,5.792744796895372e-05]]},"meta":{}}


If you see successful output, you have your first seldon-core-microservice up and running! 